In [17]:
! pip uninstall numpy -y

Found existing installation: numpy 2.1.2
Uninstalling numpy-2.1.2:
  Successfully uninstalled numpy-2.1.2


In [1]:
! pip install -r requirements.txt

In [2]:
import requests
import joblib
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from pandas import option_context
from tqdm import tqdm
from statsmodels.stats.inter_rater import aggregate_raters
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mannwhitneyu,shapiro,ttest_ind

import cliffsDelta as cd
import GenerateActivities as gat
import important_features as imf

In [ ]:
QUERY_ROOT = "https://api.github.com"
key_path = "kid.txt"
with open(key_path) as f:
    TOKEN = f.read() # write your GitHub API key here
HEADERS={'Authorization':'token '+TOKEN}

## Section 3

### Question 1

#### Write an automated Python script to identify all GitHub bot actors in the list of contributors, by querying the GitHub REST API users endpoint and extracting the required information to make this decision. 

#### More information: You can search for the "type" key in the obtianed JSON result, If it is "Bot", then it is a bot actor, if it is "User", then it is an account.

In [27]:
# 1. Read the given <sample>.csv file into a pandas DataFrame (df).
# 2. Make sure there are 2 columns, one column name is "contributor" that has the name of the contributor, 
#    and another column name is "bothunter_type" that you will use later on in this assignment.
# 3. Display the DataFrame.

# YOUR CODE BELOW

data = pd.read_csv("Contributors/sample12.csv")

print(data.shape[1] == 2 and (data.columns[0], data.columns[1]) == ('contributor', 'bothunter_type'))
data

True


,contributor,bothunter_type
0,Bastien-mva,Human
1,FrancescAlted,Human
2,GPUtester,Human
3,JuliaTagBot,Bot
4,MartinBeseda,Human
5,Schefflera-Arboricola,Human
6,Will-Tyler,Human
7,andamian,Human
8,aylward,Human
9,bioc-workshop-dev,Human


In [38]:
# 1. Store the names of the contributors in a list
# 2. print the list

contributors = data['contributor'].to_list()
contributors

['Bastien-mva',
 'FrancescAlted',
 'GPUtester',
 'JuliaTagBot',
 'MartinBeseda',
 'Schefflera-Arboricola',
 'Will-Tyler',
 'andamian',
 'aylward',
 'bioc-workshop-dev',
 'brichet',
 'codecov-commenter',
 'conda-bot',
 'cstjean',
 'dask-bot',
 'dependabot[bot]',
 'dschult',
 'francesco-ballarin',
 'galaxyproject-sentryintegration[bot]',
 'github-merge-queue[bot]',
 'hackmd-hub[bot]',
 'henryiii',
 'jariji',
 'jorainer',
 'kdheepak',
 'lldelisle',
 'martinfleis',
 'mitzimorris',
 'nekrut',
 'oxinabox',
 'prbzrg',
 'renovate[bot]',
 'review-notebook-app[bot]',
 'samaloney',
 'spinkney',
 'tardis-bot',
 'tomelse',
 'transifex-integration[bot]',
 'wtbarnes',
 'zeptodoctor']

In [97]:
# 1. Using a for loop, iterate over each contributor in the list and query the GitHub Users API.
#    You can use "query = f'{QUERY_ROOT}/users/{contributor}'", where QUERY_ROOT is defined at the beginning of this notebook 
#    and 'contributor' is each individual contributor from the list
# 2. Get the response using 'response = requests.get(query, headers=HEADERS)'.
# 3. convert the response to JSON using 'json_response = response.json()'.
# 4. Iterate over each JSON response and get the value of the 'type' key. If it is "Bot" then the contributor is a bot actor, 
#    if "User" then the contributor is an account. You should boolean values to indicate if the contributor is a bot actor (True) or User/Organisation (False)
# 5. Save these results in list of dictionary of the form [{'contributor': <contributor name>, 'app': <boolean value>}, {...}, {...}, {...}].
#    Lets call this list as "app_dict"
# 6. Finally convert this list of dictionary to DataFrame by writing 'pd.DataFrame.from_dict(app_dict)'
# 7. Display the DataFrame. This should have two columns - contributor and app

app_dict = list()
for con in contributors:
    query = f'{QUERY_ROOT}/users/{con}'
    response = requests.get(query, headers=HEADERS)
    json_response = response.json()
    print(json_response)
    app_dict.append({'contributor' : con, 'user_type': json_response['type']})
# YOUR CODE BELOW
app_dict = pd.DataFrame.from_dict(app_dict)
app_dict

{'login': 'Bastien-mva', 'id': 71976811, 'node_id': 'MDQ6VXNlcjcxOTc2ODEx', 'avatar_url': 'https://avatars.githubusercontent.com/u/71976811?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/Bastien-mva', 'html_url': 'https://github.com/Bastien-mva', 'followers_url': 'https://api.github.com/users/Bastien-mva/followers', 'following_url': 'https://api.github.com/users/Bastien-mva/following{/other_user}', 'gists_url': 'https://api.github.com/users/Bastien-mva/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/Bastien-mva/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/Bastien-mva/subscriptions', 'organizations_url': 'https://api.github.com/users/Bastien-mva/orgs', 'repos_url': 'https://api.github.com/users/Bastien-mva/repos', 'events_url': 'https://api.github.com/users/Bastien-mva/events{/privacy}', 'received_events_url': 'https://api.github.com/users/Bastien-mva/received_events', 'type': 'User', 'user_view_type': 'public', 'site_admin': 

,contributor,user_type
0,Bastien-mva,User
1,FrancescAlted,User
2,GPUtester,User
3,JuliaTagBot,User
4,MartinBeseda,User
5,Schefflera-Arboricola,User
6,Will-Tyler,User
7,andamian,User
8,aylward,User
9,bioc-workshop-dev,User


### Question 2

#### Add a new 'actor' column in the CSV file to store this information. You can write True if it is a bot actor and False if it is not.

#### Report on the total number of GitHub bot actors, and User accounts present in the list of accounts of your dataset.

In [98]:
# Merge the app DataFrame to df by writing 'pd.merge(df, <app df>, on='contributor'). This is similar to SQL join on primary key 'contributor'.
# The resultant df should have 3 columns - contributor, bothunter_type and user_type.

# YOUR CODE BELOW
df = pd.merge(data, app_dict, on='contributor')
df

,contributor,bothunter_type,user_type
0,Bastien-mva,Human,User
1,FrancescAlted,Human,User
2,GPUtester,Human,User
3,JuliaTagBot,Bot,User
4,MartinBeseda,Human,User
5,Schefflera-Arboricola,Human,User
6,Will-Tyler,Human,User
7,andamian,Human,User
8,aylward,Human,User
9,bioc-workshop-dev,Human,User


### Question 3

#### For each bot actor, write their purpose, the task that they automate by looking at their GitHub profile, homepage, information on the GitHub Marketplace, or other documentation that you can find online.


In [99]:
for i, row in df.iterrows():
    if row.user_type: continue
    print(row.contributor)
    

# dependabot[bot] : Automated dependency updates built into GitHub
# galaxyproject-sentryintegration[bot] : Galaxy is an open, web-based platform for data-intensive research. Ansible playbook for managing Galaxy infrastructure
# github-merge-queue[bot] : GitHub action to help merge PRs in a queueing manner 
# hackmd-hub[bot] : HackMD lets the community collaborate on your documentation in real-time
# renovate[bot] : Dependency Automation service by Mend.io
# review-notebook-app[bot] : Code Review & Collaboration for Jupyter Notebooks
# transifex-integration[bot] : Invoke the Transifex CLI as part of a Github action 

## Section 4

### Question 1

#### Write an automated script to use BIMBAS (Bot Identification Model Based on Activity Sequences) to obtain a prediction of the GitHub contributor type (Bot or Human) for the contributors present in your dataset.

#### Folow the steps given in each cell to use BIMBAS and obtain predictions.

### Read all the events and select the events performed by the contributors present in the given set

In [100]:
# 1. Read the csv file of events (all_events.csv) provided along with this notebook
# 2. Convert created_at column to datetime format 
#    One possible way is to use lambda function: "events.assign(created_at=lambda d: pd.to_datetime(d.created_at, unit='ms'))"
# 3. Get the contributors provided to you in a list or use the contributors list that you created in Section 3 Question 1. 
#    e.g., ['contributor1', 'contributor2', 'contributor3',...]
# 4. Select the events performed by these contributors alone. Do not consider all the events. You should consider only the events
#    performed by the contributors provided to you. Hint: you can use the df.query function
# 5. Display the considered events.

# YOUR CODE BELOW

shrek = pd.read_csv("all_events.csv").assign(created_at=lambda d: pd.to_datetime(shrek.created_at, unit='ms'))
# shrek = pd.to_datetime(shrek['created_at'], unit='ms')

shrek = shrek[shrek['login'].isin(contributors)]
shrek

,Unnamed: 0,event_id,event_type,login,repository,created_at,action,PR_number,state,PR_node_id,...,comment_node_id,merged,ref,ref_type,issue_number,issue_node_id,issue_closed_at,tag_name,release_node_id,org
29,29,39747452010,IssueCommentEvent,JuliaTagBot,SciML/CommonWorldInvalidations.jl,2024-07-01 00:07:04,created,NaN,closed,NaN,...,IC_kwDOMOEgaM6DD0Tk,NaN,NaN,NaN,5.0,I_kwDOMOEgaM6N6wg8,2024-06-28T18:51:28Z,NaN,NaN,SciML
163,163,39747766396,CreateEvent,dependabot[bot],SciML/EasyModelAnalysis.jl,2024-07-01 00:34:53,NaN,NaN,NaN,NaN,...,NaN,NaN,dependabot/github_actions/actions/add-to-proje...,branch,NaN,NaN,NaN,NaN,NaN,SciML
164,164,39747766717,PullRequestEvent,dependabot[bot],SciML/EasyModelAnalysis.jl,2024-07-01 00:34:55,opened,261.0,open,PR_kwDOI1YYHs50AC5Y,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SciML
232,232,39747938052,PushEvent,zeptodoctor,JuliaLang/docs.julialang.org,2024-07-01 00:48:33,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JuliaLang
369,369,39749032522,CreateEvent,renovate[bot],OSGeo/grass,2024-07-01 02:04:56,NaN,NaN,NaN,NaN,...,NaN,NaN,renovate/pdal-pdal-2.x,branch,NaN,NaN,NaN,NaN,NaN,OSGeo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358230,358230,42413693698,PullRequestEvent,dependabot[bot],dask/distributed,2024-09-30 21:55:05,opened,8880.0,open,PR_kwDOAocYk859LMPw,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dask
358273,358273,42414119186,CreateEvent,dependabot[bot],dask/dask,2024-09-30 22:16:32,NaN,NaN,NaN,NaN,...,NaN,NaN,dependabot/github_actions/JamesIves/github-pag...,branch,NaN,NaN,NaN,NaN,NaN,dask
358274,358274,42414119674,PullRequestEvent,dependabot[bot],dask/dask,2024-09-30 22:16:34,opened,11408.0,open,PR_kwDOAbcwm859LTCS,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dask
358282,358282,42414205596,CreateEvent,dependabot[bot],scientific-python/upload-nightly-action,2024-09-30 22:21:10,NaN,NaN,NaN,NaN,...,NaN,NaN,dependabot/github_actions/actions-aa3137d618,branch,NaN,NaN,NaN,NaN,NaN,scientific-python


### Execute BIMBAS to obtain the predictions

In [101]:
# Do not modify this cell
# Use the following function the code below to obtain your predictions

def execute_bimbas(contributors_list, selected_events):
    '''
    args: contributors_list (list) - list of contributors in the dataset provided to you
          selected_events (DataFrame) - DataFrame of events performed by the considered contributors
    return: bimbas_prediction (DataFrame) - DataFrame of preditions along with the confidence in prediction for each contributor
    '''
    result=pd.DataFrame()
    temp_list = []
    bimbas = joblib.load('bimbas.joblib')
    date_limit = pd.to_datetime(selected_events.created_at.max()) + pd.DateOffset(-90)
    
    for contributor in tqdm(contributors_list):
        # c = contributors[contributor]
        activities = gat.activity_identification(selected_events.query('login==@contributor and created_at>=@date_limit'))
        activity_features = (
                            imf.extract_features(activities)
                            .set_index([[contributor]])
                            )
        # features = pd.concat([features,activity_features])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            probability = bimbas.predict_proba(activity_features)[0][1]
        if(probability <= 0.5):
            contributor_type = 'Human'
        else:
            contributor_type = 'Bot'
        confidence = (abs(probability - 0.5)*2).round(3)
    
        pred = activity_features.set_index([[contributor]]).assign(
            prediction=contributor_type,
            confidence = confidence,
            )
        temp_list.extend([{'contributor':contributor,'bimbas_type':contributor_type,'confidence':confidence}])
    
    bimbas_prediction = pd.DataFrame.from_dict(temp_list)

    return(bimbas_prediction)

In [102]:
# Write your code here to call execute_bimbas function. 
# Pass the contributors list and the filtered events DataFrame as argument to "execute_bimbas" function
# Prediction returned by BIMBAS will have "contributor", "bimbas_type" and "confidence"
# Note: Print the prediction provided by BIMBAS - else this cell will not be graded.

# YOUR CODE BELOW

temp = execute_bimbas(contributors, shrek)

100%|██████████| 40/40 [00:16<00:00,  2.40it/s]


### Create a column in your predictions DataFrame to have predictions provided by BIMBAS. 
### Now your DataFrame should have the following columns - contributor, bothunter_type, app, bimbas_type, confidence 

### Merging bothunter and bimbas predictions

In [103]:
# Merge the predictions provided by BIMBAS to the resultant dataset of Section 3. This final DataFrame should have the following columns - 
# contributor, bothunter_type, user_type, bimbas_type, and confidence.

df = pd.merge(df, temp, on='contributor')
df

,contributor,bothunter_type,user_type,bimbas_type,confidence
0,Bastien-mva,Human,User,Bot,0.862
1,FrancescAlted,Human,User,Human,0.913
2,GPUtester,Human,User,Bot,0.841
3,JuliaTagBot,Bot,User,Bot,0.345
4,MartinBeseda,Human,User,Bot,0.883
5,Schefflera-Arboricola,Human,User,Human,0.952
6,Will-Tyler,Human,User,Human,0.930
7,andamian,Human,User,Human,0.917
8,aylward,Human,User,Human,0.851
9,bioc-workshop-dev,Human,User,Bot,0.887


### Question 2
#### Using Cohen's Kappa compute and report the interrater agreement score between the labels computed by both the bot identification approaches. Mention your interpretation of Cohen's Kappa

In [105]:
# Hint: use the cohen_kappa_score library
# YOUR CODE BELOW

cohen_kappa_score(df['bothunter_type'].to_list(), df['bimbas_type'].to_list())

# score of 0.5789 is a meh score

np.float64(0.5789473684210527)

### Question 3
#### Determine the final type of each contributor. Whenever user_type column has the value "User" check if both bimbas_type and bothunter_type give the same prediction, then consider it as your final prediction in "acc_type" column.
#### For the contributors that have different predictions, i.e. bimbas_type is not same as bothunter_type, then make a manual verification and consider that as the type for that contributor. Add an extra column to the DataFrame ﬁle called 'manual' that has the label determined by you. 
#### For manual verification you can make use the GitHub UI and the GitHub API https://api.github.com/users/username

#### Whenever user_type column has the value "Bot", directly write your final acc_type as "Bot Actor"

#### More information:
For each contributor, you can look at their activities in GitHub UI, look at their latest events using GitHub Events API - https://api.github.com/users/<contributor>/events, to make a decision on their type.

In addition, you can also write a very small reason why do you think they are bot or human (e.g., same activity at regular interval so Bot, files committed and code modified looks like Human, comments look like Human, comment look like Bot, test report looks automated so Bot, and so on....). This will be useful to answer your next question.

In [ ]:
# Write the code for your manual decision here
# example : 
# manual_list_dict = [{'contributor': '<contributor name>', 'manual': '<your prediction>'}, {....}, {....}, {....}]
# pd.DataFrame.from_dict(manual_list_dict)


user = df.query("user_type == 'User' and bothunter_type == bimbas_type")
man_to_review = df.query("user_type == 'User' and bothunter_type != bimbas_type")
bot = df.query("user_type == 'Bot'")

man_to_review

# Bastien-mva is a human because the commit are random and nice pic bro
# GPUtester is bot because it does not follow anyone
# MartinBeseda is a human because it has random repositories (and not many)
# bioc-workshop-dev is a human because not much activity
# conda-bot is a bot because it say so + name
# tardis-bot is a bot because *beep* *boop* (and huge amount of commit on 2 repo only)
# tomelse is a human because not mutch commit, has a few repositories
# zeptodoctor maybe a forgotten bot account or a dead human (okay maybe not dead but not using this account)



,contributor,bothunter_type,user_type,bimbas_type,confidence
0,Bastien-mva,Human,User,Bot,0.862
2,GPUtester,Human,User,Bot,0.841
4,MartinBeseda,Human,User,Bot,0.883
9,bioc-workshop-dev,Human,User,Bot,0.887
12,conda-bot,Human,User,Bot,0.841
35,tardis-bot,Human,User,Bot,0.331
36,tomelse,Human,User,Bot,0.742
39,zeptodoctor,Human,User,Bot,0.937


In [12]:
# Write your code below here to determine the final type of contributor.
# add column named 'acc_type' and write your final decision in it. Final decision is majority of three types - bimbas_type, bothunter_type, manual.


### Question 4
#### Study and report the purpose of these identified bots and humans in the repository assigned to you.
#### For example, the purpose can be based on but not limited to - 
##### 1) the type of activities that they are performing (releasing a version on every Sunday, updating the documentation), 
##### 2) type of comments they are posting (reviewing code, test summary report), 
##### 3) when are they triggered (when a new PR is created, when someone in the project ask the bot to merge the code) and so on.  
#### Note: 
##### The purpose can be other than what is provided in examples above.

In [ ]:
# no i dont want to

## Section 5

### Question 1
#### Use the filtered events file that has the events performed by the contributors provided to you
#### Group each event into the following four categories

Issues: IssueCommentEvent, IssuesEvent  
Pull Requests: PullRequestEvent, PullRequestReviewCommentEvent  
Commits: CommitCommentEvent, PushEvent  
Repository: CreateEvent, DeleteEvent, ForkEvent, GollumEvent, MemberEvent, PublicEvent, ReleaseEvent, SponsorshipEvent, WatchEvent  

#### Hint:
1. Add a column called event_group that mentions which event group does that event belong to. Each event (row) should correspond to an event group.  
2. Then perform groupby on ['login','event_group'],  
3. use .agg (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) to count the number of events performed by each contributor in each group,  
4. use pivot with the required arguments (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html). An example is given [here](#pivot_example) in question 2(b), and
5. Reset index and rename axis with None
7. Finally merge it with your 'acc_type' field from the DataFrame you created in Section4, drop contributor, and fillna with 0
8. The final DataFrame should have the following columns - login, PR, commit, issue, repo, acc_type

The final DataFrame should be looking like the following  
![](event_group.png)

In [76]:
# In the resultant DataFrame - each row should corespond to a contributor, and the columns should have all the event groups  
# and the type of the contributor (that you decided in the previous DataFrame) and the values should be the number 
# of events of that event event group the contriubutor has performed.

issue_group = ['IssueCommentEvent', 'IssuesEvent']
pr_group = ['PullRequestEvent', 'PullRequestReviewCommentEvent']
commit_group = ['CommitCommentEvent', 'PushEvent']
repo_group = ['CreateEvent', 'DeleteEvent', 'ForkEvent', 'GollumEvent', 'MemberEvent', 'PublicEvent', 'ReleaseEvent', 'SponsorshipEvent', 'WatchEvent']

In [96]:
# YOUR CODE BELOW
def sortin(typ):
    typ = typ.event_type
    if typ in issue_group:
        return 'Issues'
    if typ in pr_group:
        return 'Pull Requests'
    if typ in commit_group:
        return 'Commits'
    if typ in repo_group:
        return 'Repository'
    
donkey = shrek.apply(sortin, axis=1)
drag = shrek.assign(event_group=donkey.values).agg('event_group')
display(drag)
res = drag.groupby('login')['event_group']
display(res)

29               Issues
163          Repository
164       Pull Requests
232             Commits
369          Repository
              ...      
358230    Pull Requests
358273       Repository
358274    Pull Requests
358282       Repository
358283    Pull Requests
Name: event_group, Length: 8585, dtype: object

KeyError: 'login'

In [20]:
# YOUR CODE BELOW


### Question 2 (a)

#### Compute the median number of events per event group for Bot+Bot actors and Humans and write in DataFrame.

Row should correspond to type (Bot_BotActor and Human), Column should have Event group name and the values should be the median value of Bot_BotActor or Human for that particular event group. An example is given below

In [15]:
# For example:
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot_app
1,event_group1,val2,Human
2,event_group2,val3,Bot_app
3,event_group2,val4,Human
4,event_group3,val5,Bot_app
5,event_group3,val6,Human
6,event_group4,val7,Bot_app
7,event_group5,val8,Human


In [22]:
# YOUR CODE BELOW

### Question 2 (b)

Plot a heatmap of the DataFrame using seaborn - 
1. First convert the dataframe to the required format using pivot, example is given below
2. plot using seaborn - sns.heatmap(df_medians, annot=True, vmin=0, vmax=300, cmap="crest"). More details: https://seaborn.pydata.org/generated/seaborn.heatmap.html)

#### pd.pivot example:
<a id='pivot_example'></a>

In [17]:
# Main DataFrame
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot
1,event_group1,val2,Human
2,event_group2,val3,Bot
3,event_group2,val4,Human
4,event_group3,val5,Bot
5,event_group3,val6,Human
6,event_group4,val7,Bot
7,event_group5,val8,Human


In [18]:
# pivot the main DataFrame
df_medians.pivot(index='acc_type', columns='event_group', values='median')#.reset_index().rename_axis(None,axis=1)

event_group,event_group1,event_group2,event_group3,event_group4,event_group5
acc_type,,,,,
Bot,val1,val3,val5,val7,NaN
Human,val2,val4,val6,NaN,val8


In [25]:
# YOUR CODE BELOW

In [26]:
# YOUR CODE BELOW


#### What is the difference that you observe between Bots+Bot actors and Humans?

#### What is the differnce that you observe between Event groups?

#### What is the difference that you observe between Bots+Bot actors and Humans and Event groups all considering at the same time?

### Question 3

#### Create boxen plots to visualise the distribution of number of events in each event group. 
#### For more information you can visit - https://seaborn.pydata.org/generated/seaborn.boxenplot.html#seaborn.boxenplot 
#### You should highlight the data points that correspond to bots using a stripplot in seaborn. https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### Interpret the results of the visualisation.

In [27]:
# YOUR CODE BELOW - Visualize number of events from pull request event group


In [28]:
# YOUR CODE BELOW - Visualise number of events from issue event group


In [29]:
# YOUR CODE BELOW - Visualise number of events from commit event group


In [30]:
# YOUR CODE BELOW - Visualise number of events from repo event group


### Question 4.1

#### Statistical identify whether number of events in each event group is normally distributed or not.
#### Null hypothesis - $H_0$: Sample comes from the data that has normal distribution.
#### Use Shapiro-Wilk test for this purpose. Use the p-value with a threshold of 0.05 to determine whether $H_0$ can be rejected with statistical significance or not. 

#### Use shapiro from scipy.stats to perform this test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

In [ ]:
# YOUR CODE BELOW

### Question 4.2
#### Determine if there is any statistical diﬀerence in number of events between the identified event groups. Perform this test on all pairs of event groups. 
#### Null hypothesis - $H_0$: Any two event group come from the same population.
#### If $H_0$ is rejected in Shapiro-Wilk test (at least for one of the two event groups considered for test), use the Mann-Whitney U statistical test for this purpose. If $H_0$ is not rejected in Shapiro-Wilk test, use the independent t-test for this purpose.  
#### In any case, use the p-value with a threshold of 0.01 to determine whether $H_0$ can be rejected with statistical significance. 

#### Use mannwhitneyu from scipy.stats to perform Mann-Whitney U test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) 
#### or 
#### ttest_ind from scipy.stats to perform independent t-test https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html.

#### More information:

You can pass the following arguments to mannwhitneyu from scipy.stats - (method='exact', nan_policy='omit'). For ttest_ind you can use - (nan_policy='omit')

In [31]:
# YOUR CODE BELOW

### Question 4.3
#### Each time you reject the null hypothesis $H_0$, quantify the effect size of the diﬀerence between the groups using cliﬀ’s delta ($\delta$). 
#### To calculate cliﬀ’s delta, you can pass the list of values to cliﬀ delta.py file given in the repository. E.g., cliffsDelta.cliffsDelta(list of values, list of values). This will return the effect size.
#### Refer to the table given in the TP document and mention your interpretation (negligible, small, medium, large).

In [ ]:
# YOU CODE BELOW